In [37]:
from lyricsgenius import Genius
import requests
import urllib.request
import urllib.parse
import json
from bs4 import BeautifulSoup
from search import * # other Python file with functions

In [76]:
### a new client_access_token might be needed to be retrieved for successful scraping 
base = "https://api.genius.com/"
client_access_token = "9aNRd35L6ys6_JB5G0V164oZL-6KA7h0KYiigCLJRkVnYevkup10KRYlMFwxpUQm"

In [39]:
### https://github.com/ANOVAProjectUSYD/lyrics-analysis
### code is old so we had to update the way we retrieve the lyrics since the html file layout has changed since.
### added few functions as well and modified some of the functionality to fit our purposes better. 
def connect_lyrics(song_id):
    '''Constructs the path of song lyrics.'''
    url = "songs/{}".format(song_id)
    data = get_json(url)

    # Gets the path of song lyrics
    path = data['response']['song']['path']
    print(path)
    return path


def retrieve_lyrics(song_id):
    '''Retrieves lyrics from html page.'''
    path = connect_lyrics(song_id)

    URL = "http://genius.com" + path
    page = requests.get(URL)

    # Extract the page's HTML as a string
    html = BeautifulSoup(page.text, "html.parser")

    # Scrape the song lyrics from the HTML
    output = ""
    for lyrics in html.select('div[class^="Lyrics__Container"]'):
        output += lyrics.get_text(strip=True, separator='\n')
    return output


def get_song_id(artist_id, page_limit, songs_limit):
    '''Get all the song id from an artist.'''
    current_page = 1
    next_page = True
    songs = [] # to store final song ids

    while next_page and current_page <= page_limit:
        path = "artists/{}/songs/".format(artist_id)

        page_songs = get_json(path)['response']['songs']
        if page_songs:
            # Add all the songs of current page
            songs += page_songs
            print("Artist ID {} Page {} finished scraping".format(artist_id, current_page))
            # Increment current_page value for next loop
            current_page += 1

        else:
            # If page_songs is empty, quit
            next_page = False

    # Get all the song ids, excluding not-primary-artist songs.
    # print(json.dumps(songs, indent = 4))
    
    songs_id_list = []
    i = 1
    for song in songs:
        if song["primary_artist"]["id"] == artist_id and i <= songs_limit:
            songs_id_list.append(song["id"])
            i += 1      
    print(songs_id_list)
    return songs_id_list


def get_song_information(song_ids, amount):
    '''Retrieve meta data about a song.'''
    # initialize a dictionary.
    song_list = {}
    print("Scraping song information")
    for i, song_id in enumerate(song_ids):
        print("id:" + str(song_id) + " start. ->")

        path = "songs/{}".format(song_id)
        try:
            data = get_json(path=path)["response"]["song"]
        except:
            data = {"title" : "failed to fetch" + song_id}
        song_list.update({
        i: {
            "title": data["title"] if data else "",
            "album": data["album"]["name"] if data["album"] else "<single>",
            "album_cover" : data["album"]["cover_art_url"] if data["album"] and data["album"]["cover_art_url"] else "",
            "genius_album_id": data["album"]["id"] if data["album"] else "none",
            "release_date": data["release_date"] if data["release_date"] else "unidentified",
            "featured_artists":
                [feat["name"] if data["featured_artists"] else "" for feat in data["featured_artists"]],
            "featured_artists_pics":
                [feat["image_url"] if data["featured_artists"] else "" for feat in data["featured_artists"]],
            "producer_artists":
                [feat["name"] if data["producer_artists"] else "" for feat in data["producer_artists"]],
            "writer_artists":
                [feat["name"] if data["writer_artists"] else "" for feat in data["writer_artists"]],
            "primary_artist_picture":
                data["primary_artist"]["image_url"] if data["primary_artist"]["image_url"] else "",
            "genius_track_id": song_id}
        })
        print("-> id:" + str(song_id) + " is finished. \n")
        if i > amount:
            print("Scrapped lyrics of " , i, "songs. Done.")
            break
            
    return song_list

def get_artist_id_via_genius(artist_name):
    genius = Genius()
    artist = genius.search_artist(artist_name, max_songs=0, sort="title")
    try:
        if artist.name == artist_name:
            results = []
            results.append(artist.name)
            results.append(artist.id)
            return results
    except:
        print('no result found')

## Pre-scraping processing:
### We load the artists list collected from Spotfiy and
### use geniuslyrics to find the artist id on genius.com

In [ ]:
## Pre-scraping

##Finding the artist ids and saving them in a csv file

import csv
from typing import Set, Dict
# def get_ids_from_csv(filepath):
filepath = 'artist_data.csv'
file = open(filepath, encoding='utf-8')
csvreader = csv.reader(file)
artists_list = []
for artist in csvreader:
    artists_list.append(artist)
    
artists_id_list = []    
for artist in artists_list:
    result = get_artist_id_via_genius(artist[0])
    try:
        if result[0].lower() == artist[0].lower():
            artists_id_list.append(result)
            print("Found a match!")
    except:
        print("No Found")

    ################# save output
with open("artists_id_list.csv", "w", newline="", encoding='utf8') as f:
    writer = csv.writer(f)
    writer.writerows(artists_id_list)

SyntaxError: '(' was never closed (2863057677.py, line 26)

## Batch-Scraping using multithreading

In [9]:
######################################################
###### Scraping the lyrics using multithreading ######
######################################################

import concurrent.futures
import csv
from ipywidgets import IntProgress
from IPython.display import display
import time

##This function was originally written to batch scrap all artists
##Since we want concurrent multithreading, we must have changed it
##so it only processes one artist
def get_them_songs(artist_name, artist_id, songs_per_artist = 15, pages_to_scrape = 5):
    result = {}
    song_ids = get_song_id(artist_id, pages_to_scrape, songs_per_artist)
    song_list = get_song_information(song_ids, songs_per_artist)
    song_lyrics = [retrieve_lyrics(song_id) for song_id in song_ids]
    for i, lyrics in enumerate(song_lyrics):
        song_list[i].update({"song_id" : song_ids[i], "lyrics" : lyrics})
        # if i >= songs_per_artist:
        #     break
    result.update({
        artist_name : song_list
    }) # each index (with respect to ordercontains a list of songs
    return result

###########
##Reading the artists_id_list and dumping into a dictionary
artists_id_list = {}
with open('artists_id_list.csv') as csv_file:
    reader = csv.reader(csv_file)
    for row in reader:
        name = row.pop(0)
        artist_id = int(row.pop(0))
        artists_id_list.update({name: artist_id})

## getting an list for names & list for ids
names = []
ids = []
for key, value in artists_id_list.items():
    names.append(key)
    ids.append(value)

##Initializing final dictionary for the output
final = {}
##We can limit how many names & ids we want to scrap
scrap_amount = 10
names = names
ids = ids
##Initializing a bar to see progress
f = IntProgress(min=0, max=len(names)) # instantiate the bar
display(f) # display the bar
##initiaiting a ThreadPoolExecutor to multithread the scraping process
##Workers return their results to the final dictionary once done scraping the songs of an artist
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    for result in executor.map(get_them_songs, names, ids):
        final.update(result)
        f.value += 1

IntProgress(value=0, max=782)

Artist ID 47931 Page 1 finished scraping
Artist ID 403521 Page 1 finished scraping
Artist ID 25183 Page 1 finished scraping
Artist ID 37917 Page 1 finished scraping
Artist ID 12800 Page 1 finished scraping
Artist ID 18832 Page 1 finished scraping
Artist ID 12202 Page 1 finished scraping
Artist ID 11959 Page 1 finished scraping
Artist ID 403521 Page 2 finished scraping
Artist ID 47931 Page 2 finished scraping
Artist ID 37917 Page 2 finished scraping
Artist ID 403521 Page 3 finished scraping
Artist ID 18832 Page 2 finished scraping
Artist ID 47931 Page 3 finished scraping
Artist ID 12800 Page 2 finished scraping
Artist ID 25183 Page 2 finished scraping
Artist ID 12202 Page 2 finished scraping
Artist ID 403521 Page 4 finished scraping
Artist ID 37917 Page 3 finished scraping
Artist ID 11959 Page 2 finished scraping
Artist ID 18832 Page 3 finished scraping
Artist ID 403521 Page 5 finished scraping
[1802123, 2238998, 1802112, 1802123, 2238998, 1802112, 1802123, 2238998, 1802112, 1802123, 22

### Saving into 3 different file types (txt, csv, json)
### Checking that the JSON file is loadable back into a dictionary
*When loading the JSON file into a dictionary, the indices of the songs are being interpreted as strings*

**TODO:** *find a solution for this issue that doesn't require iterating over all song's indices and convert them to int*

In [75]:
###save the results###
export_to_3_types(final, "final")

In [72]:
### Save the results into 3 different file types
### JSON seems to be the one we're actually looking
### CSV is a big-NONO, but saved anyway

import json
import csv

## saving as different file types, only JSON is really needed.
def export_to_3_types(filename, filename_save):
    with open(filename_save + '.txt', 'w', encoding='utf8') as f:
        f.write(str(filename))

    # Save the dictionary as .json
    with open(filename_save + '.json', 'w', encoding='utf8') as f:
        json.dump(filename, f)

    # Save the dictionary as .csv
    with open(filename_save + '.csv', 'w', encoding='utf8') as f:
        w = csv.DictWriter(f, filename.keys())
        w.writeheader()
        w.writerow(filename)

In [73]:
### checking that the json file was saved correctly and can be loaded into a dictionary ###
with open('final.json', 'r') as f:
    load_me = json.load(f)

# create a dictionary from the loaded json file
load_final = dict(load_me)

### Loading into dictionary works well

## deleting empty artists
i = 0
j = 0
empties = 0
to_be_deleted = []
for name, idd in load_final.items():
    for song_index in load_final[name]:
        i += 1
        j += 1
    if j == 0:
        to_be_deleted.append(name)
        empties += 1
    j = 0


print('Before:',len(load_final))
for artist in to_be_deleted:
    load_final.pop(artist)
print('After:',len(load_final))
export_to_3_types(load_final, 'final_clean')

782
700


In [92]:
## After loading the JSON into the dictionary
## it seems like the indices of the songs is being read as a string
## following turns it back into int, need to find a better solution for that
for artist in load_final:
    load_final[artist] = {int(idx):value for idx, value in load_final[artist].items()}

load_final['Kollegah'][0]

{'title': '1001 Nacht',
 'album': 'Kollegah',
 'album_cover': 'https://images.genius.com/07e288ab9cef1fc67d4eab53f47737d8.1000x1000x1.jpg',
 'genius_album_id': 11399,
 'release_date': '2008-08-29',
 'featured_artists': [],
 'featured_artists_pics': [],
 'producer_artists': ['Rizbo'],
 'writer_artists': ['Rizbo', 'Kollegah'],
 'primary_artist_picture': 'https://images.genius.com/851cc7ad19da70bb8b380be671ef83d6.1000x1000x1.png',
 'genius_track_id': 52276,
 'song_id': 52276,
 'lyrics': "[Songtext zu „1001 Nacht“]\n[Intro]\nJebiga, Johnny, du bist ein dreckiger Johnny (Hehe)\nRäusper, Räusper, sag' ich mal so ins Blaue hinein\nYeah, yeah, Rene, ey, pass auf\n[Bridge]\nYeah, sieh, wie ich massenweise Zaster schmeiße (Ey)\nFlatternde Scheine regnen herab auf Pflastersteine (Ey)\nHustler-Style, ich ernte neidische Blicke\nDenn 20-Zoll-Felgen sind am Reifen wie Früchte (Ey)\n[Hook]\nUnd ich mach' ein paar Tausend in einer Nacht\n1001 Gramm verkaufen in einer Nacht\n1001 Scherben Schaufensters

In [6]:
##Batch Scraping - NO LONGER USED

from typing import Dict
import csv

songs_limit = 10

artists_id_list : Dict[str, int] = {}
with open('artists_id_list.csv') as csv_file:
    reader = csv.reader(csv_file)
    for row in reader:
        name = row.pop(0)
        artist_id = int(row.pop(0))
        artists_id_list.update({name: artist_id})


# for artist, artist_id in artists_id_list.items():
#     print(artist, artist_id)

## Batch Scraping from a given list of artist ids.
# def batch_scrap_lyrics(artists_id_list):
final = {}
for artist_name, artist_id in artists_id_list.items():
    song_ids = get_song_id(artist_id, 5, songs_limit)
    song_list = get_song_information(song_ids, songs_limit)
    song_lyrics = [retrieve_lyrics(song_id) for song_id in song_ids]
    for i, lyrics in enumerate(song_lyrics):
        song_list[i].update({"song_id" : song_ids[i], "lyrics" : lyrics})
        if i >= songs_limit:
            break
    final.update({
        artist_name : song_list
    }) # each index (with respect to ordercontains a list of songs
    


Artist ID 25183 Page 1 finished scraping


KeyboardInterrupt: 